In [1]:
import os
path = "C:/pytest/"
os.chdir(path)

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
data1_train = pd.read_csv('data1_train.csv',encoding= 'cp949')
data1_test = pd.read_csv('data1_test.csv',encoding= 'cp949')

In [4]:
data1_train_X_morphed = data1_train['텍스트']
data1_train_y = data1_train['분류1_상위']
data1_test_X_morphed = data1_train['텍스트']
data1_test_y = data1_train['분류1_상위']

In [5]:
from tqdm import tqdm

In [6]:
max_words = 10000
maxlen = 30
embedding_dim = 200

In [7]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(data1_train_X_morphed)
word_index = tokenizer.word_index

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# sequencing 
data1_train_X_sequencing = tokenizer.texts_to_sequences(data1_train_X_morphed)
# padding
data1_train_X_padding = pad_sequences(data1_train_X_sequencing, maxlen = maxlen)

In [10]:
from sklearn.preprocessing import LabelEncoder
e = LabelEncoder()
e.fit(data1_train_y)
data1_train_y_labeling = e.transform(data1_train_y)

In [11]:
print(e.classes_)

['IT_과학' '경제' '국제' '문화' '미분류' '사회' '스포츠' '정치' '지역']


In [12]:
def to_one_hot(sequences, dimension):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results

In [13]:
data1_train_y_1hot = to_one_hot(data1_train_y_labeling, len(e.classes_))

In [14]:
class_number = len(e.classes_)

In [15]:
from keras.models import Sequential
from keras import layers, regularizers
model = Sequential()
model.add(layers.Embedding(input_dim = max_words, output_dim = embedding_dim, input_length = maxlen))

model.add(layers.Conv1D(64, kernel_size = 3,activation = 'relu'))
# model.add(layers.MaxPooling1D(pool_size = 2)) # maxpooling + lstm : 82.8 / maxpooling+flatten : 84.1
# model.add(layers.Flatten())
model.add(layers.Dropout(0.4)) # conv + dropout + lstm + dropout : 84.5

model.add(layers.LSTM(64)) # 83.6

model.add(layers.Dropout(0.4))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(class_number, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 200)           2000000   
                                                                 
 conv1d (Conv1D)             (None, 28, 64)            38464     
                                                                 
 dropout (Dropout)           (None, 28, 64)            0         
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 9)                 2

In [17]:
model.compile(loss= 'categorical_crossentropy',optimizer= 'rmsprop',metrics = ['acc'])
with tf.device('/CPU:0'):
    history = model.fit(data1_train_X_padding, data1_train_y_1hot,epochs = 10, batch_size =32, validation_split= 0.3,verbose = 1)

Epoch 1/10
149/149 [==============================] - 4s 18ms/step - loss: 1.4967 - acc: 0.4651 - val_loss: 1.2524 - val_acc: 0.5515
Epoch 2/10
149/149 [==============================] - 3s 18ms/step - loss: 1.0154 - acc: 0.6262 - val_loss: 1.1048 - val_acc: 0.6030
Epoch 3/10
149/149 [==============================] - 2s 15ms/step - loss: 0.8124 - acc: 0.7087 - val_loss: 1.0764 - val_acc: 0.6398
Epoch 4/10
149/149 [==============================] - 3s 18ms/step - loss: 0.5801 - acc: 0.8056 - val_loss: 1.1574 - val_acc: 0.6727
Epoch 5/10
149/149 [==============================] - 3s 19ms/step - loss: 0.3704 - acc: 0.8849 - val_loss: 1.2094 - val_acc: 0.7041
Epoch 6/10
149/149 [==============================] - 3s 18ms/step - loss: 0.2555 - acc: 0.9251 - val_loss: 1.2084 - val_acc: 0.6992
Epoch 7/10
149/149 [==============================] - 3s 19ms/step - loss: 0.1717 - acc: 0.9485 - val_loss: 1.4802 - val_acc: 0.6953
Epoch 8/10
149/149 [==============================] - 3s 19ms/step - 

In [18]:
# data1
# sequencing
data1_test_X_sequencing = tokenizer.texts_to_sequences(data1_test_X_morphed)
# padding
data1_test_X_padding = pad_sequences(data1_test_X_sequencing, maxlen = maxlen)

In [19]:
# data1 test_y
data1_test_y_labeling= e.transform(data1_test_y)
data1_test_y_1hot = to_one_hot(data1_test_y_labeling, class_number)

In [20]:
# data1 분류1
with tf.device('/CPU:0'):
    model.evaluate(data1_test_X_padding, data1_test_y_1hot)

213/213 [==============================] - 1s 6ms/step - loss: 0.5427 - acc: 0.9065
